## Trial 1 9/20/2023

In [ ]:
import sys
from scipy.io import mmread

if len(sys.argv) != 3:
    print("Usage: python mtx_to_metis.py <input_file.mtx> <output_file.txt>")
    sys.exit(1)

input_file = sys.argv[1]
output_file = sys.argv[2]

# Read the sparse matrix from the .mtx file
matrix = mmread(input_file).tocsr()

n = matrix.shape[0]  # Number of vertices
m = matrix.nnz // 2  # Number of edges, divide by 2 for undirected graphs

# Write to the output file in METIS format
with open(output_file, 'w') as f:
    f.write(f"{n} {m}\n")  # Write the header with number of vertices and edges
    
    for i in range(n):
        # Extract the neighboring vertices for vertex i
        neighbors = matrix[i].indices + 1  # +1 as METIS format is 1-indexed
        neighbors_str = ' '.join(map(str, neighbors))
        f.write(neighbors_str + '\n')

print(f"Conversion from {input_file} to {output_file} completed successfully!")


In [1]:
import sys
from scipy.io import mmread

input_file = 'SMALL_chesapeake.mtx'
output_file = 'SMALL_metis_chesapeake.txt'

# Read the sparse matrix from the .mtx file
matrix = mmread(input_file).tocsr()

n = matrix.shape[0]  # Number of vertices
m = matrix.nnz // 2  # Number of edges, divide by 2 for undirected graphs

# Write to the output file in METIS format
with open(output_file, 'w') as f:
    f.write(f"{n} {m}\n")  # Write the header with number of vertices and edges
    
    for i in range(n):
        # Extract the neighboring vertices for vertex i
        neighbors = matrix[i].indices + 1  # +1 as METIS format is 1-indexed
        neighbors_str = ' '.join(map(str, neighbors))
        f.write(neighbors_str + '\n')

print(f"Conversion from {input_file} to {output_file} completed successfully!")


ValueError: too many values to unpack (expected 5)

## Trial 2 9/26/2023

In [1]:
import json

# Load the .mtx file
with open('SMALL_chesapeake.mtx', 'r') as mtx_file:
    mtx_data = json.load(mtx_file)

# Extract necessary information from the JSON data
num_nodes = mtx_data['blob']['rawLines'][2].split()[0]
num_edges = mtx_data['blob']['rawLines'][2].split()[1]
edges = mtx_data['blob']['rawLines'][3:]

# Create a METIS format file
with open('SMALL_chesapeake.txt', 'w') as metis_file:
    # Write the header line with the number of nodes and edges
    metis_file.write(f"{num_nodes} {num_edges}\n")
    
    # Write the edge data
    for edge in edges:
        edge_data = edge.split()[2:]  # Extract adjacency information
        metis_file.write(" ".join(edge_data) + "\n")

print("Conversion complete. METIS format file 'SMALL_chesapeake.txt' has been created.")


KeyError: 'blob'

In [3]:
def convert_json_mtx_to_metis(input_file, output_file):
    with open(input_file, 'r') as mtx_file, open(output_file, 'w') as metis_file:
        mtx_data = json.load(mtx_file)
        raw_lines = mtx_data.get('blob', {}).get('rawLines', [])

        num_nodes, num_edges = None, None
        edge_data = []

        for line in raw_lines:
            line = line.strip()
            if line.startswith("%"):  # Skip comments
                continue
            if not num_nodes:
                num_nodes, num_edges, _ = map(int, line.split())
            else:
                edge_data.append(line)

        metis_file.write(f"{num_nodes} {num_edges}\n")
        metis_file.write("\n".join(edge_data))

    print(f"Conversion complete. METIS format file '{output_file}' has been created.")

input_file = 'SMALL_chesapeake.mtx'
output_file = 'SMALL_chesapeake.txt'

# Usage: Specify the input JSON-based .mtx file and the desired output file in METIS format
convert_json_mtx_to_metis(input_file, output_file)

Conversion complete. METIS format file 'SMALL_chesapeake.txt' has been created.


## Kent's Legacy code

Whacky metadata SMALL_chesapeake.mtx format to normal mtx format

In [21]:
import json

input_file_name = "SMALL_chesapeake.mtx"  # Name of the input file containing JSON data
output_file_name = "norm_SMALL_chesapeake.mtx"  # Name of the output file to write Matrix Market data

try:
    # Read JSON data from the input file
    with open(input_file_name, 'r') as f:
        data = json.load(f)

    # Extract the raw lines containing the matrix data
    raw_lines = data.get('payload', {}).get('blob', {}).get('rawLines', [])
    format_header = f"{raw_lines[0]}\n"

    # Open the output file
    with open(output_file_name, 'w') as f:
        # Write Matrix Market format header (if you want)
        f.write(format_header)

        # Write the matrix data from the JSON to the output file
        for line in raw_lines:
            # Skip lines starting with a single '%' or '%%'
            if line.startswith('%'):
                continue
            f.write(line + '\n')

    print(f"Matrix Market file '{output_file_name}' has been written successfully.")

except json.JSONDecodeError:
    print(f"Error: The file '{input_file_name}' does not contain valid JSON data.")

except FileNotFoundError:
    print(f"Error: The file '{input_file_name}' does not exist.")


Matrix Market file 'norm_SMALL_chesapeake.mtx' has been written successfully.


## USC's conversion code